In [7]:
load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [3]:
!conda install Cython

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/johnbauer/anaconda:
#
cython                    0.26.1           py27h6a053f9_0  


In [12]:
%%cython --annotate
cdef int add(int x, int y):
    return x + y 

CompileError: command 'gcc' failed with exit status 1

In [3]:
%%cython --annotate

import cython
#cimport cpython.array

import numpy as np
cimport numpy as np

from libc.math cimport sin, cos

FLOAT64 = np.float64
ctypedef np.float64_t FLOAT64_t

def hs_lower_triangular(int dim, np.ndarray zeta):
    cdef np.ndarray L_arr = np.zeros((dim, dim), dtype=FLOAT64)
    # Memoryview of the numpy array
    cdef FLOAT64_t [:, :] L = L_arr
    cdef int r, s, j
    
    L[0,0] = 1.0
    for r in range(1, dim):
        for s in range(r):
            L[r,s] = cos(zeta[r,s])
            for j in range(s):
                L[r,s] *= sin(zeta[r,j])
        L[r,r] = 1.0
        for j in range(r):
            L[r,r] *= sin(zeta[r,j])
    return L_arr

print(hs_lower_triangular(3, np.ndarray([0.1, 0.5, 0.2])))

CompileError: command 'gcc' failed with exit status 1

In [4]:
%%cython --annotate

import cython
#cimport cpython.array

import numpy as np
cimport numpy as np

from libc.math cimport sin, cos, pi

FLOAT64 = np.float64
ctypedef np.float64_t FLOAT64_t

def HyperSphere_lower_triangular_derivative(int dim, np.ndarray zeta, int dr, int ds):
    # TODO: return a 3-d array instead of a list
    cdef int m = dim*(dim-1)/2
    cdef np.ndarray dL_arr = np.zeros((dim, dim), dtype=FLOAT64)
    cdef FLOAT64_t [:, :] dL = dL_arr
    cdef size_t s, j
    
    for s in range(dr):
        if 0 <= ds <= s:
            dL[dr,s] = cos(zeta[dr,s]) if s != ds else -sin(zeta[dr,s])
            for j in range(s):
                dL[dr,s] *= sin(zeta[dr,j]) if j != ds else cos(zeta[dr,j])
    dL[dr,dr] = 1.0 
    for j in range(dr):
        dL[dr,dr] *= sin(zeta[dr,j]) if j != ds else cos(zeta[dr,j])
    return dL_arr


Error compiling Cython file:
------------------------------------------------------------
...
FLOAT64 = np.float64
ctypedef np.float64_t FLOAT64_t

def HyperSphere_lower_triangular_derivative(int dim, np.ndarray zeta, int dr, int ds):
    # TODO: return a 3-d array instead of a list
    cdef int m = dim*(dim-1)/2
                           ^
------------------------------------------------------------

/Users/johnbauer/.ipython/cython/_cython_magic_ade426660bc72cdc4d6764e7d43c5313.pyx:15:28: Cannot assign type 'double' to 'int'


In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 24 09:16:47 2017

@author: johnbauer
"""
import numpy as np

import cython
cimport cpython.array

cimport numpy as np
from libc.math cimport sin, cos, pi

FLOAT64 = np.float64
ctypedef np.float64_t FLOAT64_t

# TODO: enforce dim*dim-1)/2 = len(theta), remove from signature
class HyperSphere(object):
    """Parameterizes the d-1-dimensional surface of a d-dimensional hypersphere
    using a lower triangular matrix with d*(d-1)/2 parameters, each in the 
    interval (0, pi).
    """
    def __init__(self, dim, zeta=[]):
        m = dim*(dim-1)//2
        self.dim = dim
        if isinstance(zeta, (list, tuple)) and len(zeta):
            assert len(zeta) == m, "Expecting {0}*({0}-1)/2 elements".format(dim)
        elif isinstance(zeta, (int, float, np.float64, np.int64)):
            zeta = [zeta]
        else:
            zeta = [pi/4.0]*m
        zeta_lt = np.zeros((dim, dim))
        # lower triangular indices, offset -1 to get below-diagonal elements
        for th, ind in zip(zeta, zip(*np.tril_indices(dim,-1))):
            zeta_lt[ind] = th
        # set the diagonal to 1
        for i in range(dim):
            zeta_lt[i,i] = 1.0
        self.zeta = zeta_lt
        self._lt = HyperSphere_lower_triangular(dim, zeta_lt)
        #self._lt = self._lower_triangular()
            
    def _lower_triangular(self):
        dim = self.dim
        zeta = self.zeta
        L = np.zeros((dim, dim), dtype=np.float64)
        L[0,0] = 1.0
        for r in range(1,dim):
            for s in range(r):
                L[r,s] = cos(zeta[r,s])
                for j in range(s):
                    L[r,s] *= sin(zeta[r,j])
            L[r,r] = 1.0
            for j in range(r):
                L[r,r] *= sin(zeta[r,j])
        return L
    
    @property
    def correlation(self):
        lt = self._lt
        return lt.dot(lt.T)

    def _lower_triangular_derivative(self):
        dim = self.dim
        zeta = self.zeta
        #dL[0,0] = 0.0
        dLstack = []
        for dr, ds in zip(*np.tril_indices(dim, -1)):
            dL = HyperSphere_lower_triangular_derivative(dim, zeta, dr, ds)
            dLstack.append(dL)
        return dLstack
    
    def _lower_triangular_derivative_original(self):
        dim = self.dim
        zeta = self.zeta
        #dL[0,0] = 0.0
        dLstack = []
        for dr, ds in zip(*np.tril_indices(dim, -1)):
            dL = np.zeros((dim, dim), dtype=np.float64)
            for s in range(dr):
                if s == ds or ds in range(s):
                    dL[dr,s] = cos(zeta[dr,s]) if s != ds else -sin(zeta[dr,s])
                    for j in range(s):
                        dL[dr,s] *= sin(zeta[dr,j]) if j != ds else cos(zeta[dr,j])
            dL[dr,dr] = 1.0 
            for j in range(dr):
                dL[dr,dr] *= sin(zeta[dr,j]) if j != ds else cos(zeta[dr,j])
#                else:
#                    # strictly speaking could skip this since initialized to zero anyway
#                    dL[r,s] = 0.0
            dLstack.append(dL)
        return dLstack
    
    def gradient(self):
        L = self._lt
        dLstack = self._lower_triangular_derivative()
        gradstack = [] 
        for dL in dLstack:
            dLLt = dL.dot(L.T)
            grad = dLLt + dLLt.T
            gradstack.append(grad)
        return gradstack
        # TODO: call np.dstack here, us np.newaxis to handle matrix mult
        # cf kernels.py
        #return np.dstack(gradstack)





In [ ]:
if __name__ == "__main__":
    
    lt = HyperSphere(2)
    print("2-parameter, lower triangular\n", lt._lower_triangular())
    
    lt1 = HyperSphere(3, [pi, pi/3, pi/6])
    print("3-parameter, lower triangular\n", lt1._lower_triangular())
    
    ltz = HyperSphere(5) #, [0.0]*10)
    print("5-parameter, lower triangular\n", ltz._lower_triangular())